# Algoritmos Genéticos resolvendo um labirinto

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/flavio-mota/si-rna-ag-2025/blob/main/AG/AG_Labirinto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

Neste notebook vamos construir, passo a passo, um Algoritmo Genético (AG) para ensinar um agente a sair de um labirinto.

A ideia é usar este exemplo como um "jogo" didático:

- Temos um labirinto em grade, com paredes e caminhos livres.
- Um agente começa em uma posição inicial e precisa chegar até a saída.
- Cada indivíduo da população do AG representa uma sequência de movimentos (Cima, Baixo, Esquerda, Direita).
- O AG vai evoluir essas sequências até encontrar caminhos cada vez melhores.

Ao longo do notebook, vamos:

1. Representar o labirinto em forma de matriz.
2. Definir os movimentos possíveis (C, B, E, D).
3. Implementar uma função de **fitness** que recompensa caminhos "inteligentes".
4. Implementar os operadores do AG (seleção, cruzamento, mutação).
5. Visualizar o caminho encontrado pelo melhor indivíduo.


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
import io
from collections import deque
from matplotlib.colors import ListedColormap

# Para reprodutibilidade dos resultados
random.seed(42)
np.random.seed(42)

## Representando o labirinto

Vamos representar o labirinto como uma matriz de 0s e 1s:

- `0` significa **caminho livre**.
- `1` significa **parede**.

Também vamos definir:

- A posição de início (`inicio`).
- A posição de objetivo (`objetivo`).

Depois disso, vamos criar uma função para visualizar o labirinto.

In [ ]:
# Para gráficos mais agradáveis
plt.rcParams["figure.figsize"] = (6, 6)

# --- Labirinto 15x15 (0 = caminho, 1 = parede) ---
labirinto = np.array([
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1], # 0
    [0,0,0,1,0,1,0,1,1,0,1,0,1,1,1], # 1
    [1,0,0,0,0,0,0,1,0,0,0,0,1,0,1], # 2
    [1,1,0,1,1,1,0,1,0,1,1,0,1,0,1], # 3
    [1,0,0,0,0,1,0,0,0,1,0,0,0,0,1], # 4
    [1,1,1,1,0,1,1,1,0,1,1,1,1,0,1], # 5
    [1,0,0,0,0,0,0,1,0,1,0,0,1,0,1], # 6
    [1,0,1,1,1,1,1,1,0,1,0,1,1,0,1], # 7
    [1,0,0,0,0,1,0,1,0,0,0,1,0,0,1], # 8
    [1,1,1,1,0,1,0,1,1,1,0,1,1,1,1], # 9
    [1,0,0,1,0,0,0,0,0,1,0,1,0,0,1], # 10
    [1,1,0,1,1,1,1,1,0,1,1,1,1,0,1], # 11
    [1,1,0,0,0,0,0,0,0,0,1,1,1,0,1], # 12
    [1,0,0,1,0,1,1,0,1,0,0,0,0,0,1], # 13
    [1,1,1,1,1,1,1,1,1,1,1,1,1,0,0]  # 14
    #0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
])

inicio = (0, 0) # linha, coluna
objetivo = (14, 14) # linha, coluna

def mostrar_labirinto():
    plt.figure(figsize=(4, 4))
    # Paredes em preto (1), caminhos em branco (0)
    plt.imshow(labirinto, cmap='gray_r')

    # Marca início (verde) e objetivo (vermelho)
    y_i, x_i = inicio
    y_g, x_g = objetivo
    plt.scatter([x_i], [y_i], marker='o')   # início
    plt.scatter([x_g], [y_g], marker='x')   # objetivo

    plt.title("Labirinto (início = bolinha, objetivo = X)")
    plt.axis('off')
    plt.show()

mostrar_labirinto()

## Codificando os movimentos em português

Vamos codificar os movimentos com letras em português:

- `C` = Cima  
- `B` = Baixo  
- `E` = Esquerda  
- `D` = Direita  

Vamos definir a lista de movimentos possíveis e a função que aplica um movimento à posição atual, respeitando as paredes e os limites do labirinto.

In [ ]:
# Movimentos possíveis
MOVIMENTOS = ['C', 'B', 'E', 'D']  # Cima, Baixo, Esquerda, Direita

def mover(posicao, passo):
    """
    Aplica um movimento (C, B, E, D) a partir da posição atual.
    Se bater em parede ou tentar sair do labirinto, permanece no lugar.
    """
    y, x = posicao

    if passo == 'C':      # Cima
        y -= 1
    elif passo == 'B':    # Baixo
        y += 1
    elif passo == 'E':    # Esquerda
        x -= 1
    elif passo == 'D':    # Direita
        x += 1

    # Garante que não sai dos limites da matriz
    y = max(0, min(y, labirinto.shape[0] - 1))
    x = max(0, min(x, labirinto.shape[1] - 1))

    # Se for parede, não anda
    if labirinto[y, x] == 1:
        return posicao

    return (y, x)

## Simulando um cromossomo (trajetória)

Cada indivíduo do AG será um vetor de movimentos, por exemplo:

`['C', 'C', 'D', 'D', 'B', 'B', ...]`

Vamos criar uma função para **simular** a execução de um cromossomo, gerando o caminho percorrido a partir do início.

Importante: para ter caminhos mais “humanos”, vamos parar a simulação assim que o agente chegar ao objetivo. Não faz sentido ele continuar andando depois de ter saído do labirinto.

In [ ]:
def simular_cromossomo(cromossomo, inicio=inicio, parar_no_objetivo=True):
    """
    Executa os movimentos do cromossomo a partir da posição de início.
    Retorna (posicao_final, caminho_percorrido).
    """
    pos = inicio
    caminho = [pos]

    for passo in cromossomo:
        pos = mover(pos, passo)
        caminho.append(pos)

        if parar_no_objetivo and pos == objetivo:
            break

    return pos, caminho


def plotar_caminho(caminho, titulo="Caminho percorrido"):
    plt.figure(figsize=(4, 4))
    plt.imshow(labirinto, cmap='gray_r')

    ys = [p[0] for p in caminho]
    xs = [p[1] for p in caminho]
    plt.plot(xs, ys, linewidth=2)

    y_i, x_i = inicio
    y_g, x_g = objetivo
    plt.scatter([x_i], [y_i], marker='o')   # início
    plt.scatter([x_g], [y_g], marker='x')   # objetivo

    plt.title(titulo)
    plt.axis('off')
    plt.show()

In [ ]:
# Exemplo de cromossomo aleatório só para ver a simulação
exemplo = [random.choice(MOVIMENTOS) for _ in range(20)]
pos_final, caminho_exemplo = simular_cromossomo(exemplo)
plotar_caminho(caminho_exemplo, titulo=f"Exemplo aleatório (final = {pos_final})")

## Função de aptidão baseada na distância real até o objetivo

Para avaliar a qualidade de um cromossomo, vamos utilizar uma função de aptidão (fitness) que considera não apenas os movimentos locais, mas também a estrutura completa do labirinto. A ideia é quantificar o quão eficaz o caminho proposto é para alcançar a saída.

Nossa função de aptidão combina quatro elementos principais:

1. **Distância real até o objetivo**  
   Vamos calcular, para cada célula, a distância mínima até o objetivo considerando apenas caminhos livres, ou seja, respeitando as paredes do labirinto.  
   Essa distância será obtida por meio de uma busca em largura (BFS) a partir da posição do objetivo, resultando em um “mapa de distâncias”.  
   Durante a simulação do cromossomo, sempre que um movimento leva o agente para uma célula com distância menor do que a anterior, interpretamos isso como progresso real e isso aumenta a pontuação.

2. **Penalização por movimentos inválidos**  
   Se o agente tentar atravessar uma parede ou sair dos limites do labirinto, ele permanece na mesma posição. Nesses casos, aplicamos uma penalização, pois esse tipo de ação não contribui para alcançar a solução.

3. **Penalização por revisitar células já visitadas**  
   Caminhos que retornam constantemente aos mesmos lugares tendem a ser ineficientes.  
   Assim, cada vez que o agente revisita uma posição, aplicamos uma penalização leve.

4. **Recompensa por alcançar o objetivo**  
   Quando o agente chega ao destino, ele recebe uma recompensa significativa, proporcional ao número de movimentos que ainda restam.  
   Isso incentiva soluções que chegam ao objetivo rapidamente, não apenas “eventualmente”.

Com esses elementos combinados, a função de aptidão orienta a evolução genética para caminhos que:

- respeitam a estrutura do labirinto,  
- evitam movimentos inúteis,  
- progridem de forma consistente até a saída,  
- e buscam alcançar o objetivo no menor número possível de passos.

A seguir, implementamos o mapa de distâncias via BFS e atualizamos a função de aptidão para utilizar essa abordagem.

In [ ]:
def calcular_distancias_labirinto(labirinto, objetivo):
    """
    Calcula a distância mínima de cada célula até o objetivo,
    considerando apenas caminhos livres (0). Células sem acesso
    ao objetivo recebem np.inf.
    """
    h, w = labirinto.shape
    dist = np.full((h, w), np.inf, dtype=float)

    fila = deque()
    y0, x0 = objetivo
    dist[y0, x0] = 0
    fila.append((y0, x0))

    # Movimentos possíveis: Cima, Baixo, Esquerda, Direita
    movimentos = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while fila:
        y, x = fila.popleft()
        for dy, dx in movimentos:
            ny, nx = y + dy, x + dx

            if 0 <= ny < h and 0 <= nx < w and labirinto[ny, nx] == 0:
                if dist[ny, nx] > dist[y, x] + 1:
                    dist[ny, nx] = dist[y, x] + 1
                    fila.append((ny, nx))

    return dist

# Calculamos uma vez o mapa de distâncias
mapa_distancias = calcular_distancias_labirinto(labirinto, objetivo)

## Visualizando o mapa de distâncias do labirinto

Como calculamos, para cada célula livre, a distância mínima até o objetivo usando BFS,
podemos representar esse resultado como um *heatmap*.

Essa visualização ajuda a entender como o algoritmo percebe a “geografia” do labirinto:

- quanto **mais claro**, mais próximo do objetivo;
- quanto **mais escuro**, mais distante;
- células **inacessíveis** recebem valor infinito e são mostradas com uma cor distinta.

Esse mapa torna evidente que certos caminhos, apesar de parecerem próximos
geometricamente, exigem trajetórias longas (ou impossível) para chegar à saída.

In [ ]:
def visualizar_mapa_distancias(mapa_distancias):
    # Copiamos o mapa para evitar alterar o original
    dist_plot = mapa_distancias.copy()

    # Substitui valores infinitos por -1 (para exibir com outra cor)
    dist_plot = np.where(np.isinf(dist_plot), -1, dist_plot)

    # Criamos uma paleta onde:
    #  - valores -1 (inacessíveis) ficam em preto
    #  - valores válidos usam uma paleta contínua (viridis)
    cmap = plt.cm.viridis
    new_cmap = ListedColormap(
        np.vstack((
            np.array([0, 0, 0, 1]),  # preto para inacessível
            cmap(np.linspace(0, 1, 255))
        ))
    )

    plt.figure(figsize=(5,5))
    img = plt.imshow(dist_plot + 1, cmap=new_cmap)  # +1 para que -1 vire 0 → preto
    plt.colorbar(img, label="Distância mínima até o objetivo")
    plt.title("Mapa de distâncias calculado por BFS")
    plt.axis("off")
    plt.show()

visualizar_mapa_distancias(mapa_distancias)


## Criando a função de fitness com a distância real do labirinto

Agora que temos o mapa de distâncias mínimas, podemos utilizá-lo diretamente na função de fitness.  
Em vez de olhar apenas para coordenadas ou aproximações geométricas, vamos comparar:

- a distância do indivíduo antes do movimento  
- a distância depois do movimento  

Assim, movimentos que realmente aproximam do objetivo ganham pontos, e movimentos que o afastam ou o levam para regiões inacessíveis são penalizados.

Essa função deixa o AG mais alinhado com a estrutura real do labirinto e melhora significativamente a qualidade dos caminhos evoluídos.

In [ ]:
def distancia_labirinto(posicao):
    """Distância mínima até o objetivo de acordo com o mapa gerado por BFS."""
    y, x = posicao
    return mapa_distancias[y, x]

def fitness(cromossomo):
    pos = inicio
    pontuacao = 0.0
    visitados = set([pos])

    for i, passo in enumerate(cromossomo):
        pos_antiga = pos
        dist_antiga = distancia_labirinto(pos_antiga)

        pos = mover(pos, passo)
        dist_nova = distancia_labirinto(pos)

        # Penalização por não se mover (parede ou limite)
        if pos == pos_antiga:
            pontuacao -= 0.5

        # Penalização por revisitar posição
        elif pos in visitados:
            pontuacao -= 0.3

        # Aproximação real do objetivo
        elif dist_nova < dist_antiga:
            pontuacao += 1.0

        # Afastamento real do objetivo
        elif dist_nova > dist_antiga:
            pontuacao -= 0.2

        # Movimento "neutro" (não aproximou nem afastou)
        else:
            pontuacao -= 0.05

        visitados.add(pos)

        # Chegou ao objetivo
        if pos == objetivo:
            restante = len(cromossomo) - (i + 1)
            pontuacao += 100.0 + 0.3 * restante
            break

    return pontuacao

In [ ]:
fitness(exemplo)

## Representando indivíduos e a população

Agora vamos definir:

- O **tamanho do cromossomo**: quantos movimentos cada indivíduo pode fazer.
- O **tamanho da população**.
- Quantas **gerações** o AG vai rodar.

Também vamos implementar funções para:

- Criar um indivíduo aleatório.
- Criar a população inicial.

In [ ]:
TAMANHO_CROMOSSOMO = 60   # número máximo de movimentos
TAMANHO_POPULACAO = 200
GERACOES = 300
TAXA_MUTACAO = 0.05

def criar_individuo():
    return [random.choice(MOVIMENTOS) for _ in range(TAMANHO_CROMOSSOMO)]

def criar_populacao():
    return [criar_individuo() for _ in range(TAMANHO_POPULACAO)]

populacao = criar_populacao()
len(populacao), len(populacao[0])

## Seleção, cruzamento e mutação

Agora definimos os operadores genéticos:

- **Seleção**: vamos usar torneio simples, escolhendo alguns indivíduos ao acaso e pegando o melhor.
- **Cruzamento (crossover)**: vamos fazer um ponto de corte e misturar as partes dos pais.
- **Mutação**: com uma pequena probabilidade, trocamos um movimento por outro aleatório.

In [ ]:
def selecionar_por_torneio(populacao, k=3):
    """Seleção por torneio: escolhe k indivíduos aleatórios e retorna o melhor."""
    escolhidos = random.sample(populacao, k)
    melhor = max(escolhidos, key=fitness)
    return melhor

def cruzamento(pai1, pai2):
    """Cruzamento de um ponto."""
    ponto = random.randint(1, TAMANHO_CROMOSSOMO - 1)
    filho1 = pai1[:ponto] + pai2[ponto:]
    filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2

def mutacao(individuo, taxa_mutacao=TAXA_MUTACAO):
    """Mutação: troca alguns movimentos aleatoriamente."""
    for i in range(len(individuo)):
        if random.random() < taxa_mutacao:
            individuo[i] = random.choice(MOVIMENTOS)
    return individuo

## Laço de evolução do Algoritmo Genético

Agora vamos juntar tudo:

- Para cada geração:
  - Avaliamos o fitness dos indivíduos.
  - Guardamos o melhor indivíduo global.
  - Criamos uma nova população por seleção, cruzamento e mutação.

Também vamos imprimir periodicamente o melhor fitness para acompanhar a evolução.

In [ ]:
melhor_global = None
fitness_melhor_global = float('-inf')

# Guardar melhores indivíduos para gerar GIF
melhores_geracoes = {}  # geração → cromossomo

historico_melhor = []

for geracao in range(GERACOES):
    # Avalia população atual
    avaliacoes = [(ind, fitness(ind)) for ind in populacao]
    melhor_da_geracao, fit_melhor_geracao = max(avaliacoes, key=lambda x: x[1])

    # Atualiza melhor global
    if fit_melhor_geracao > fitness_melhor_global:
        melhor_global = melhor_da_geracao[:]
        fitness_melhor_global = fit_melhor_geracao

    historico_melhor.append(fitness_melhor_global)

    # Exibe progresso de tempos em tempos
    if (geracao + 1) % 10 == 0 or geracao == 0:
      print(f"Geração {geracao+1:4d} | Melhor fitness global: {fitness_melhor_global:.2f}")

    melhores_geracoes[geracao + 1] = melhor_global[:]  # salva para o GIF

    # Cria nova população
    nova_populacao = []

    while len(nova_populacao) < TAMANHO_POPULACAO:
        pai1 = selecionar_por_torneio(populacao)
        pai2 = selecionar_por_torneio(populacao)

        filho1, filho2 = cruzamento(pai1, pai2)
        filho1 = mutacao(filho1)
        filho2 = mutacao(filho2)

        nova_populacao.append(filho1)
        if len(nova_populacao) < TAMANHO_POPULACAO:
            nova_populacao.append(filho2)

    populacao = nova_populacao

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(historico_melhor)
plt.xlabel("Geração")
plt.ylabel("Melhor fitness global")
plt.title("Evolução do fitness ao longo das gerações")
plt.grid(True)
plt.show()

## Visualizando a solução encontrada

Agora vamos:

1. Simular o **melhor indivíduo global** encontrado.
2. Visualizar o caminho no labirinto.
3. Mostrar o cromossomo em termos de movimentos (C, B, E, D) e quantidade de passos usados.

In [ ]:
pos_final, caminho_melhor = simular_cromossomo(melhor_global)

print("Posição final do melhor indivíduo:", pos_final)
print("Chegou ao objetivo?", "Sim" if pos_final == objetivo else "Não")
print("Tamanho do caminho percorrido:", len(caminho_melhor))
print("Cromossomo (movimentos):")
print(','.join(melhor_global))

plotar_caminho(caminho_melhor, titulo="Melhor caminho encontrado pelo AG")

## (Opcional) Geração de GIF da evolução

Se você quiser gerar um GIF mostrando, geração após geração, o melhor caminho e a curva de fitness, pode reaproveitar a lógica anterior e salvar um frame por geração.

Este trecho é mais "engenharia" do que conceito de AG, então pode ser mostrado rapidamente ou deixado como extra.

In [ ]:
def gerar_frame_pil(caminho, geracao):
    """Gera um frame (imagem PIL) da evolução a partir do caminho do AG."""
    fig, ax = plt.subplots(figsize=(4, 4))

    ax.imshow(labirinto, cmap='gray_r')

    ys = [p[0] for p in caminho]
    xs = [p[1] for p in caminho]
    ax.plot(xs, ys, linewidth=2)

    # início e objetivo
    ax.scatter([inicio[1]], [inicio[0]], marker='o')
    ax.scatter([objetivo[1]], [objetivo[0]], marker='x')

    ax.set_title(f"Geração {geracao}")
    ax.axis('off')

    # Converte a figura para imagem PIL (sem precisar mexer com array)
    buffer = io.BytesIO()
    fig.savefig(buffer, format='png', dpi=80, bbox_inches='tight')
    buffer.seek(0)
    img = Image.open(buffer).convert("RGB")

    plt.close(fig)
    return img

In [ ]:
frames = []

for geracao, cromossomo in melhores_geracoes.items():
    _, caminho = simular_cromossomo(cromossomo)
    img = gerar_frame_pil(caminho, geracao)
    frames.append(img)

caminho_gif = "evolucao_labirinto.gif"

# Salva como GIF animado usando somente PIL
frames[0].save(
    caminho_gif,
    save_all=True,
    append_images=frames[1:],
    optimize=False,
    duration=100,  # duração de cada frame em ms
    loop=0         # loop infinito
)

print("GIF gerado:", caminho_gif)